In [38]:
import time
import pandas as pd
with open('input.txt') as f:
    lines = f.read().splitlines()

## Part 1

In [39]:
pd.set_option('display.max_rows', 1000)
starttime = time.perf_counter_ns()
# create matrix with all points
df = pd.DataFrame([list(line) for line in lines])

# first let's expand the universe
# rows containing only '.' are duplicated
# columns containing only '.' are duplicated

# Duplicate rows
# This could be cahnged to be faster by just checking where the '#' are and only then
# duplicate all rows and columns we need
new_rows = []
for i in range(len(df)):
    new_rows.append(df.iloc[i])
    if not df.iloc[i].str.contains('#').any():
        # duplicate row
        new_rows.append(df.iloc[i])
# create new DataFrame
df = pd.DataFrame(new_rows).reset_index(drop=True)
# Duplicate columns
new_cols = pd.DataFrame()
for col in df.columns:
    new_cols = pd.concat([new_cols, df[col]], axis=1)
    if not df[col].str.contains('#').any():
        # duplicate column
        new_cols = pd.concat([new_cols, df[col]], axis=1)

# Reset column names
new_cols.columns = range(new_cols.shape[1])

# Identify the galaxies
galaxies = [(i, j) for i in range(len(new_cols)) for j in range(len(new_cols.columns)) if new_cols.iloc[i, j] == '#']

# now we need to find the shortest path between all pairs of nodes
total_weight = 0
visited = set()
for galaxy1 in galaxies:
    for galaxy2 in galaxies:
        if galaxy1 != galaxy2 and (galaxy1, galaxy2) not in visited:
            visited.add((galaxy1, galaxy2))
            visited.add((galaxy2, galaxy1))
            #path = nx.shortest_path(universe, galaxy1, galaxy2)
            # calcualte the length by subtracting the coordinates
            path = abs(galaxy1[0] - galaxy2[0]) + abs(galaxy1[1] - galaxy2[1])
            total_weight += path
print("Length total: ", total_weight)
print("Time taken: ", round((time.perf_counter_ns() - starttime)/1000000, 3), "ms")

Length total:  9599070
Time taken:  465.128 ms


## Part 2

In [40]:
pd.set_option('display.max_rows', 1000)
starttime = time.perf_counter_ns()
# create matrix with all points
df = pd.DataFrame([list(line) for line in lines])
mrows, mcols = [], []

# Replace rows
for i in range(len(df)):
    if not df.iloc[i].str.contains('#').any():
        # Replace row with a row full of '0'
        df.iloc[i] = ['0'] * len(df.columns)
        mrows.append(i)
# Replace columns
for col in df.columns:
    if not df[col].str.contains('#').any():
        # Replace column with a column full of '0'
        df[col] = ['0'] * len(df)
        mcols.append(col)

# Identify the galaxies
galaxies = [(i, j) for i in range(len(df)) for j in range(len(df.columns)) if df.iloc[i, j] == '#']

# now we need to find the shortest path between all pairs of nodes
total_length = 0
visited = set()
for galaxy1 in galaxies:
    for galaxy2 in galaxies:
        if galaxy1 != galaxy2 and (galaxy1, galaxy2) not in visited:
            visited.add((galaxy1, galaxy2))
            visited.add((galaxy2, galaxy1))
            #path = nx.shortest_path(universe, galaxy1, galaxy2)
            # calcualte the length by subtracting the coordinates
            path = abs(galaxy1[0] - galaxy2[0]) + abs(galaxy1[1] - galaxy2[1])
            
            """
            Instead of checking all columns and rows for '0', we can save all rows and columns' indexes when we create them
            and then check if the path has any row with that index
            """
            
            for i in mrows:
                if galaxy1[0] < i < galaxy2[0] or galaxy2[0] < i < galaxy1[0]:
                    path += 1000000-1
            for i in mcols:
                if galaxy1[1] < i < galaxy2[1] or galaxy2[1] < i < galaxy1[1]:
                    path += 1000000-1
            
            total_length += path
print("Length total: ", total_length)
print("Time taken: ", round((time.perf_counter_ns() - starttime)/1000000, 3), "ms")

Length total:  842645913794
Time taken:  511.075 ms
